In [26]:
import pandas as pd
import numpy as np
df = pd.read_csv('./Building_Permits.csv')

/var/folders/pn/9t8hjz551fxgxlw6qx40xhl40000gn/T/ipykernel_9350/2597645805.py:3: DtypeWarning: Columns (22,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./Building_Permits.csv')


In [27]:
# Looks like we have a lot of work to do

df.isnull().sum()

Permit Number                                 0
Permit Type                                   0
Permit Type Definition                        0
Permit Creation Date                          0
Block                                         0
Lot                                           0
Street Number                                 0
Street Number Suffix                      49183
Street Name                                   0
Street Suffix                               611
Unit                                      42665
Unit Suffix                               49161
Description                                  62
Current Status                                0
Current Status Date                           0
Filed Date                                    0
Issued Date                                1968
Completed Date                            20098
First Construction Document Date           1977
Structural Notification                   48073
Number of Existing Stories              

In [28]:
date_columns = ['Permit Creation Date', 'Current Status Date', 'Filed Date', 'Issued Date',
                 'Completed Date', 'First Construction Document Date', 'Permit Expiration Date']
df[date_columns] = df[date_columns].astype('datetime64[ns]')

In [29]:
df['Days_to_Issue'] = (df['Issued Date'] - df['Filed Date']).dt.days

Challenges:


1. Can you try predicting permit issue times for various permit types? Which ones matter more?
2. Can you suggest which is the best week day to visit Department of Building Inspections, based on this data?
3. Can you conclude anything on the city's developments based on this data?

In [30]:
fire_only_permit_counts = df['Fire Only Permit'].value_counts()
structural_notification_counts = df['Structural Notification'].value_counts()
site_permit_counts = df['Site Permit'].value_counts()

print(fire_only_permit_counts)
print(structural_notification_counts)
print(site_permit_counts)

Fire Only Permit
Y    4611
Name: count, dtype: int64
Structural Notification
Y    1625
Name: count, dtype: int64
Site Permit
Y    1113
Name: count, dtype: int64


In [31]:
df['Fire Only Permit'] = df['Fire Only Permit'].map({'Y': True, np.nan: False})
df['Structural Notification'] = df['Structural Notification'].map({'Y': True, np.nan: False})
df['Site Permit'] = df['Site Permit'].map({'Y': True, np.nan: False})

print(df['Fire Only Permit'].value_counts())
print(df['Structural Notification'].value_counts())
print(df['Site Permit'].value_counts())

Fire Only Permit
False    45087
True      4611
Name: count, dtype: int64
Structural Notification
False    48073
True      1625
Name: count, dtype: int64
Site Permit
False    48585
True      1113
Name: count, dtype: int64


In [32]:
# in uploading file we saw an error for columns 22 and 32. And moreover, they are completely empty. Let's get rid of them 

columns_to_drop = ['TIDF Compliance', 'Voluntary Soft-Story Retrofit']
df.drop(columns=columns_to_drop, inplace=True)

In [33]:
duplicates = df[df.duplicated()]
num_duplicates = duplicates.shape[0]
print(f"Number of Duplicate Rows: {num_duplicates}")

Number of Duplicate Rows: 0


In [34]:
# Like I said, it is rather a gamble and affecting accuracy of information, so let's make a copy for this model

def fill_missing_values_all_columns(df, group_columns):
    df_copy = df.copy()

    def fill_missing(x):
        mode_values = x.mode()
        return x.fillna(mode_values[0] if not mode_values.empty else x)

    for target_column in df_copy.columns:
        if target_column not in group_columns:
            df_copy[target_column] = df_copy.groupby(group_columns)[target_column].transform(fill_missing)

    return df_copy

group_columns = ['Street Name', 'Block']
df_filled = fill_missing_values_all_columns(df, group_columns)
missing_values = df_filled.isnull().sum()
missing_values = missing_values.sort_values(ascending=False)

/var/folders/pn/9t8hjz551fxgxlw6qx40xhl40000gn/T/ipykernel_9350/3292237043.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return x.fillna(mode_values[0] if not mode_values.empty else x)
/var/folders/pn/9t8hjz551fxgxlw6qx40xhl40000gn/T/ipykernel_9350/3292237043.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return x.fillna(mode_values[0] if not mode_values.empty else x)
/var/folders/pn/9t8hjz551fxgxlw6qx40xhl40000gn/T/ipykernel_9350/3292237043.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will 

In [35]:
df_filled.to_csv('result.csv', index=False)